<a href="https://colab.research.google.com/github/arses-ui/Forecast/blob/main/forecast_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install scikit-learn
#pip install numpy
#pip install matplolib
#pip install pandas
#pip install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-learn

<font size=5> Splitting the data into train and test

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


#reading the dataset
df = pd.read_csv("/content/drive/MyDrive/nepse_full_1year.csv")
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

#splitting the dataset for train and test
split_date = "2025-06-20"
train_df = df[df["date"] <= split_date]
test_df  = df[df["date"] > split_date]


/tmp/ipython-input-3729601771.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
/tmp/ipython-input-3729601771.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


<font size=5>Removing companies with way too little data

In [ ]:
window_size=7
valid_companies = []
for symbol, group in train_df.groupby("symbol"):
    if len(group) >= window_size + 1:
        valid_companies.append(symbol)
    else:
        print(f"Skipping {symbol}: only {len(group)} rows")

train_df = train_df[train_df["symbol"].isin(valid_companies)]
test_df  = test_df[test_df["symbol"].isin(valid_companies)]


Skipping ACLBSLP: only 1 rows
Skipping ALBSLP: only 4 rows
Skipping ALICLP: only 2 rows
Skipping BFCPO: only 6 rows
Skipping CBBLPO: only 1 rows
Skipping CYCLP: only 1 rows
Skipping CZBILP: only 7 rows
Skipping EDBLPO: only 1 rows
Skipping FMDBLP: only 2 rows
Skipping GILBPO: only 1 rows
Skipping GMFILP: only 1 rows
Skipping GUFLPO: only 4 rows
Skipping HBLPO: only 3 rows
Skipping ICFCPO: only 1 rows
Skipping IGIPO: only 3 rows
Skipping JFLPO: only 1 rows
Skipping KMCDBP: only 3 rows
Skipping LBBLPO: only 1 rows
Skipping MATRIP: only 7 rows
Skipping MBLPO: only 5 rows
Skipping MDBPO: only 1 rows
Skipping MLBBLP: only 1 rows
Skipping MLBLPO: only 1 rows
Skipping MLBSLP: only 1 rows
Skipping MSLBP: only 7 rows
Skipping NABBCP: only 5 rows
Skipping NABILP: only 1 rows
Skipping NFSPO: only 1 rows
Skipping NICLPO: only 1 rows
Skipping NIFRAP: only 1 rows
Skipping NLICLP: only 3 rows
Skipping NLICP: only 4 rows
Skipping NMBPO: only 6 rows
Skipping NMLBBLP: only 5 rows
Skipping RLFLPO: only 1

<font size=5> Function to build sequences to feed the model as training data

In [ ]:
feature_cols = ["open", "high", "low", "close", "volume", "ma_5", "volatility_10"]
def build_sequences(group, window_size=7):
    X, y = [], []
    data = group[feature_cols].values
    labels = group["target"].values
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(labels[i+window_size])
    return np.array(X), np.array(y)

window_size = 7

<font size=5>Scaling the data

In [ ]:
#Initialize a scaler for the training split
scaler = MinMaxScaler()
train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])

#Using the scaler that learned on training split for  test split
test_df[feature_cols] = scaler.transform(test_df[feature_cols])

Saving the scaled datat

In [ ]:
import joblib
scaler_filename = "scaler.joblib"
joblib.dump(scaler, scaler_filename)

['scaler.joblib']

<font size=5>Building the sequences

In [ ]:
# Training
X_tr_list, y_tr_list = [], []
for symbol, group in train_df.groupby("symbol"):
  X_sym, y_sym = build_sequences(group, window_size=7)
  X_tr_list.append(X_sym)
  y_tr_list.append(y_sym)
X_train = np.concatenate(X_tr_list, axis=0)
y_train = np.concatenate(y_tr_list, axis=0)

# Testing
X_te_list, y_te_list = [], []
for symbol, group in test_df.groupby("symbol"):
  if len(group) >= window_size + 1:
    X_sym, y_sym = build_sequences(group, window_size=7)
    X_te_list.append(X_sym)
    y_te_list.append(y_sym)
X_test = np.concatenate(X_te_list, axis=0)
y_test = np.concatenate(y_te_list, axis=0)


<font size=6> Defining the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model =  Sequential([
    LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=1, activation= "sigmoid")
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=["accuracy", "Precision", "Recall"]
 )

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


<font size=5> Training the model

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - Precision: 0.4188 - Recall: 0.0178 - accuracy: 0.5627 - loss: 0.6857
Epoch 2/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.5652 - loss: 0.6850
Epoch 3/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.5683 - loss: 0.6839
Epoch 4/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.5667 - loss: 0.6845
Epoch 5/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.5660 - loss: 0.6846
Epoch 6/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.5656 - loss: 0.6848
Epoch 7/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.5676 - loss: 0.6841
Epoch 8/100
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - Preci

In [ ]:
testPredict = model.predict(X_test)
print(testPredict)

353/353 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
[[0.47844577]
 [0.43478587]
 [0.4578876 ]
 ...
 [0.43250388]
 [0.43586054]
 [0.44150126]]


#Saving the weights of the trained model

In [ ]:
model.save_weights('/content/drive/MyDrive/forecast_model.weights.h5')